<a href="https://colab.research.google.com/github/yuna1998/datamining/blob/main/stacked_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Stacked regression with Boston data

In [1]:
library(MASS)

In [2]:
install.packages("caret")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’, ‘iterators’, ‘gower’, ‘ipred’, ‘timeDate’, ‘foreach’, ‘plyr’, ‘ModelMetrics’, ‘reshape2’, ‘recipes’, ‘pROC’




In [3]:
install.packages("caretEnsemble")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘pbapply’, ‘gridExtra’




In [6]:
install.packages("doParallel")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [7]:
library(caret)
library(caretEnsemble)
library(doParallel)

Loading required package: lattice

Loading required package: ggplot2

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘caretEnsemble’


The following object is masked from ‘package:ggplot2’:

    autoplot


Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [8]:
nc = detectCores()

In [9]:
registerDoParallel(nc)

In [10]:
nc

[1] 2

In [11]:
n = dim(Boston)[1]

In [12]:
n

[1] 506

In [13]:
set.seed(1)

In [15]:
train = sample(n, n/2)

In [16]:
X_train = Boston[train, -14]

In [17]:
y_train = Boston[train, 14]

In [18]:
X_test = Boston[-train, -14]

In [19]:
y_test= Boston[-train, 14]

In [20]:
my_control = trainControl(method = "cv", number = 10, savePredictions = "final", allowParallel =TRUE )

In [23]:
set.seed(2)

In [ ]:
model_list = caretList(X_train, y_train, methodList = c("lm", "rf", "xgbTree", "xgbLinear", "svmRadial"),
preProcess = c("center", "scale"), trConrol = my_control)

Warning message in trControlCheck(x = trControl, y = target):
“trControl$savePredictions not 'all' or 'final'.  Setting to 'final' so we can ensemble the models.”
Warning message in trControlCheck(x = trControl, y = target):
“indexes not defined in trControl.  Attempting to set them ourselves, so each model in the ensemble will have the same resampling indexes.”


In [ ]:
model_list$lm
model_list$rf
model_list$xgbTree
model_list$xgbLinear
model_list$svmRadial

In [ ]:
rs = resample(model_list)
rs$values
dotplot(rs, metric = "RMSE")
modelCor(rs) #model correlations if high then not so much imporvement by ensemble

In [ ]:
#test RMSE
p_lm = predeict.train(model_list$lm, newdata = X_test)
p_rf = predeict.train(model_list$rf, newdata = X_test)
p_xgbTree = predeict.train(model_list$xgbTree, newdata = X_test)
p_xgbLinear = predeict.train(model_list$xgbLinear, newdata = X_test)
p_svmRadial = predeict.train(model_list$svmRadial, newdata = X_test)

In [ ]:
RMSE(p_lm, y_test)
RMSE(p_rf, y_test)
RMSE(p_xgbTree, y_test)
RMSE(p_xgbLinear, y_test)
RMSE(p_svmRadial, y_test)

In [ ]:
#stacked regression using a linear model
set.seed(3)
ens1 = caretEnsemble(model_list, metric = "RMSE", trConrol = my_control)
print(ens1)
summary(ens1)

In [ ]:
p_ens1 = predict(ens1, newdata = X_test)
results = data.frame(
    lm = RMSE(p_lm, y_test),
    rf = RMSE(p_rf, y_test),
    xgbTree = RMSE(p_xgbTree, y_test),
    xgbLinear = RMSE(p_xgbLinear, y_test),
    svmRadial = RMSE(p_svmRadial, y_test) ,
    stacked_regression = RMSE(p_ens1, y_test)
)


In [ ]:
#other meta learner  : elastic net : mixture of ridge and LASSO
set.seed(4)
ens2 = caret$Stack(model_list, method = "glmnet", trControl = my_control,
matric = "RMSE")
ens2

In [ ]:
p_ens2 = predict(ens2, newdata = X_test)
RMSE(p_ens2, y_test)
results$SSL = RMSE(p_ens2, y_test)
results

In [ ]:
set.seed(4)
ens3 = caret$Stack(model_list, method = "gbm", trConrol = my_contol, metric = "RMSE")
ens3
p_ens3 = predict(ens3, newdata = X_test)
RMSE(p_ens3, y_test)

In [ ]:
set.seed(4)
ens3 = caret$Stack(model_list, method = "rf", trConrol = my_contol, metric = "RMSE")
ens3
p_ens3 = predict(ens3, newdata = X_test)
RMSE(p_ens3, y_test)